In [92]:
import os
from operator import itemgetter

def _walklevel(some_dir, level=1):
    some_dir = some_dir.rstrip(os.path.sep)
    assert os.path.isdir(some_dir)
    num_sep = some_dir.count(os.path.sep)
    for root, dirs, files in os.walk(some_dir):
        yield root, dirs, files
        num_sep_this = root.count(os.path.sep)
        if num_sep + level <= num_sep_this:
            del dirs[:]

In [170]:
from collections import namedtuple


def orginise_into_recordings(todo):
    RecordingPair = namedtuple('RecordingPair', 'pre, cno')
    recordings = set(['_'.join(x.split('_')[:-1]) for x in todo])
    recording_collection = {}
    for recording in recordings:
        matches = list(filter(lambda x: recording in x, todo))
        assert len(matches) == 1 or len(matches) == 2
        print(matches)
        pre = list(filter(lambda x: 'pre' in x.lower(), matches))[0]
        cno = list(filter(lambda x: 'cno' in x.lower(), matches))[0]
        assert pre
        recording_collection[recording] = RecordingPair(pre=pre, cno=cno)
    return recording_collection
            

In [185]:
def get_files_todo(dat_file_dir, continuous_dir, verbose):

    def _rename_to_standard(paths):
        ''' Make
        .../continuous/GATD_01_2018-10-12_17-38-33_PRE ----> GATD_01_2018-10-12_PRE

        '''
        base_names = list(map(os.path.basename, paths))
        nice_names = list(map(lambda x: '_'.join(
            itemgetter(0, 1, 2, 4)(x.split('_'))), base_names))
        return nice_names

    try:
        done_files = get_subfolders(parent=dat_file_dir,
                                    containing_filetype='.dat',
                                    verbose=verbose)
    except AssertionError:  # triggers when cat dat file dir not found
        if verbose:
            print('''Could not find dat file directory
                Creating new directory {}'''.format(dat_file_dir))
        os.mkdir(dat_file_dir)
        done_files = []

    continuous_dirs = get_subfolders(parent=continuous_dir,
                                     containing_filetype='.continuous',
                                     verbose=verbose)

    dat_base = list(map(os.path.basename, done_files))
    c_formatted = _rename_to_standard(continuous_dirs)

    if not dat_base:
        todo = continuous_dirs
    else:
        for path in dat_base:
            todo = list(filter(lambda x: path not in x,
                               continuous_dirs))
    return todo

In [186]:
def get_subfolders(parent, containing_filetype=None, verbose=True):

    def _walklevel(some_dir, level=1):
        some_dir = some_dir.rstrip(os.path.sep)
        assert os.path.isdir(some_dir)
        num_sep = some_dir.count(os.path.sep)
        for root, dirs, files in os.walk(some_dir):
            yield root, dirs, files
            num_sep_this = root.count(os.path.sep)
            if num_sep + level <= num_sep_this:
                del dirs[:]

    def _has_ext(path, ext):
        if '.' not in ext:
            ext = ''.join(['.', ext])
        return bool(glob(os.path.join(path, ''.join(['*', ext]))))

    try:
        paths = [x[0]
                 for x in _walklevel(parent, level=1) if os.path.isdir(x[0])]
        if parent in paths:
            del paths[paths.index(parent)]
    except AssertionError:
        if verbose:
            print('Could not find {} dir'.format(parent))
        raise

    if containing_filetype:
        f = partial(_has_ext, ext=containing_filetype)
        paths = list(filter(f, paths))

    return paths

In [195]:
dfd = '/media/ruairi/SD_BCK/GAT_DREADD/dat_files'
cd ='/media/ruairi/SD_BCK/GAT_DREADD/continuous'

todo = get_files_todo(dat_file_dir=dfd, continuous_dir=cd, verbose=True)

In [217]:
todo

['/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_01_2018-10-12_17-38-33_PRE',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_01_2018-10-12_18-40-03_CNO',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_02_2018-10-13_12-40-56_PRE',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_02_2018-10-13_13-41-28_CNO',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_03_2018-10-13_15-59-35_PRE',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_03_2018-10-13_16-59-53_CNO',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_04_2018-10-14_12-36-18_PRE',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_04_2018-10-14_13-38-21_CNO',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_05_2018-10-14_15-54-41_PRE',
 '/media/ruairi/SD_BCK/GAT_DREADD/continuous/GATD_05_2018-10-14_16-55-49_CNO']

In [ ]:
def orginise_into_recordings(todo):
    def _rename_to_standard(paths):
        ''' Make
        .../continuous/GATD_01_2018-10-12_17-38-33_PRE ----> GATD_01_2018-10-12_PRE

        '''
        base_names = list(map(os.path.basename, paths))
        nice_names = list(map(lambda x: '_'.join(
            itemgetter(0, 1, 2, 4)(x.split('_'))), base_names))
        return nice_names

    RecordingPair = namedtuple('RecordingPair', 'pre, cno')
    recordings = set(['_'.join(x.split('_')[:-1]) for x in todo])
    recording_collection = {}
    for recording in recordings:
        matches = list(filter(lambda x: recording in x, todo))
        assert len(matches) == 1 or len(matches) == 2
        print(matches)
        pre = list(filter(lambda x: 'pre' in x.lower(), matches))[0]
        cno = list(filter(lambda x: 'cno' in x.lower(), matches))[0]
        assert pre
        recording_collection[recording] = RecordingPair(pre=pre, cno=cno)
    return recording_collection

In [208]:
class Recording:
    
    def __init__(self, name, pre, cond):
        self.name = name
        self.pre = pre
        self.cond = cond
    
    def has_cond(self):
        return bool(self.cond)

In [229]:
def organise_into_objects(todo):
    out = []
    bases = list(map(os.path.basename, todo))
    recordings = set(['_'.join(x.split('_')[:-2]) for x in bases])
    for recording in recordings:

        try:
            pre = list(filter(lambda x: recording in x and 'pre' in x.lower(), todo))[0]
            pre = os.path.basename(pre)
        except IndexError:
            print('No pre file found for recording: {}'.format(recording))
            raise
        try:
            cond = list(filter(lambda x: recording in x and 'cno' in x.lower(), todo))[0]
            cond = os.path.basename(pre)
        except IndexError:
            print('No condition file found for recording {}. Continuing anyway'.format(recording))
            cond = None
        
        pre = os.path.basename(pre)
        out.append(Recording(name=recording, pre=pre, cond=cond))
    return out

In [235]:
os.path.basename(organise_into_objects(todo)[1].pre)


'GATD_05_2018-10-14_15-54-41_PRE'

In [231]:
rec1.cond

In [236]:
x = None

'this {}'.format(x)

'this None'